# Recommender Code 

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
import findspark
findspark.init("/usr/local/spark")

In [2]:
from pyspark.sql import SparkSession

In [3]:
# May take awhile locally
spark = SparkSession.builder.appName("RecommenderSys").getOrCreate()

In [4]:
import os
os.getenv('PYTHON_SPARK')

'/home/pasharma/workspace/CodingChills/PySpark'

In [5]:
data_dir = os.path.join(os.getenv('PYTHON_SPARK'),'Data')

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [7]:
data = spark.read.csv(os.path.join(data_dir,'movielens_ratings.csv'),
                      inferSchema=True,
                      header=True)

In [8]:
data.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [9]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [10]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [11]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [12]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [13]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|     5| 0.99025565|
|     31|   3.0|     8|   3.343186|
|     31|   2.0|    25|-0.12700242|
|     31|   1.0|    24|  2.7396104|
|     31|   1.0|     0|  2.5677032|
|     85|   1.0|    13|  0.1665169|
|     85|   5.0|    16|  0.5152595|
|     85|   5.0|     8|  2.7591612|
|     65|   1.0|    22| 0.18542361|
|     65|   2.0|     5| 0.18782169|
|     65|   2.0|    15|  0.9607363|
|     65|   1.0|     4|   1.622212|
|     65|   1.0|    24|  0.9742992|
|     53|   1.0|     6|  1.2698808|
|     78|   1.0|    12| 0.45186743|
|     78|   1.0|    24|   1.308828|
|     78|   1.0|     2|  0.9985246|
|     34|   1.0|    16|  3.0988994|
|     34|   1.0|    17|  1.7332095|
|     34|   1.0|     4|  2.6329005|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8021387813169387


The RMSE described our error in terms of the stars rating column.

In [16]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [17]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     18|    11|
|     22|    11|
|     23|    11|
|     30|    11|
|     41|    11|
|     47|    11|
|     62|    11|
|     66|    11|
|     75|    11|
|     76|    11|
|     94|    11|
+-------+------+



In [19]:
reccomendations = model.transform(single_user)

In [20]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     30|    11| 5.5675454|
|     22|    11| 4.5861464|
|     18|    11| 3.8149993|
|     62|    11| 3.5673265|
|     75|    11| 2.9757023|
|     41|    11| 2.7788675|
|     23|    11| 2.7422822|
|     66|    11|  2.489182|
|      0|    11|  2.105777|
|     94|    11| 1.7494895|
|     76|    11| 1.2437519|
|     47|    11|-1.8881688|
+-------+------+----------+



## Problem Statement
The whole world seems to be hearing about your new amazing abilities to analyze big data and build useful systems for them! You've just taken up a new contract with a new online food delivery company. This company is trying to differentiate itself by recommending new meals to customers based off of other customers likings.

Can you build them a recommendation system?

Your final result should be in the form of a function that can take in a Spark DataFrame of a single customer's ratings for various meals and output their top 3 suggested meals. For example:

Best of luck!

** *Note from Jose: I completely made up this food data, so its likely that the actual recommendations themselves won't make any sense. But you should get a similar output to what I did given the example customer dataframe* **

In [38]:
import numpy as np
import pandas as pd
df = pd.read_csv(os.path.join(data_dir,'movielens_ratings.csv'))
df['mealskew'] = df['movieId'].apply(lambda id: 9999 if id > 31 else id)

In [37]:
mealmap = { 2. : "Chicken Curry",   
           3. : "Spicy Chicken Nuggest",   
           5. : "Hamburger",   
           9. : "Taco Surprise",  
           11. : "Meatloaf",  
           12. : "Ceaser Salad",  
           15. : "BBQ Ribs",  
           17. : "Sushi Plate",  
           19. : "Cheesesteak Sandwhich",  
           21. : "Lasagna",  
           23. : "Orange Chicken",
           26. : "Spicy Beef Plate",  
           27. : "Salmon with Mashed Potatoes",  
           28. : "Penne Tomatoe Pasta",  
           29. : "Pork Sliders",  
           30. : "Vietnamese Sandwich",  
           31. : "Chicken Wrap",  
           9999: "Cowboy Burger",   
           4. : "Pretzels and Cheese Plate",   
           6. : "Spicy Pork Sliders",  
           13. : "Mandarin Chicken PLate",  
           14. : "Kung Pao Chicken",
           16. : "Fried Rice Plate",  
           8. : "Chicken Chow Mein",  
           10. : "Roasted Eggplant ",  
           18. : "Pepperoni Pizza",  
           22. : "Pulled Pork Plate",   
           0. : "Cheese Pizza",   
           1. : "Burrito",   
           7. : "Nachos",  
           24. : "Chili",  
           20. : "Southwest Salad",  
           25.: "Roast Beef Sandwich"}

In [39]:
df['meal_name'] = df['mealskew'].map(mealmap)

In [40]:
df.to_csv(os.path.join(data_dir,'Meal_Info.csv'),index=False)

In [25]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [41]:
data = spark.read.csv(os.path.join(data_dir, 'Meal_Info.csv'),inferSchema=True,header=True)

In [42]:
(training, test) = data.randomSplit([0.8, 0.2])

In [46]:
training.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)
 |-- mealskew: integer (nullable = true)
 |-- meal_name: string (nullable = true)



In [47]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="mealskew", ratingCol="rating")
model = als.fit(training)

In [48]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

predictions.show()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+-------+------+------+--------+--------------------+-----------+
|movieId|rating|userId|mealskew|           meal_name| prediction|
+-------+------+------+--------+--------------------+-----------+
|     31|   4.0|    12|      31|        Chicken Wrap| 0.49936217|
|     31|   1.0|    13|      31|        Chicken Wrap|  1.4362943|
|     31|   1.0|     5|      31|        Chicken Wrap|  2.2971683|
|     31|   1.0|    24|      31|        Chicken Wrap|   1.520163|
|     31|   1.0|    29|      31|        Chicken Wrap|   1.332109|
|     28|   2.0|    15|      28| Penne Tomatoe Pasta| 0.25552803|
|     26|   1.0|    23|      26|    Spicy Beef Plate|    1.42039|
|     27|   1.0|    26|      27|Salmon with Mashe...|  2.1934903|
|     27|   1.0|     1|      27|Salmon with Mashe...|-0.67579937|
|     27|   3.0|    24|      27|Salmon with Mashe...|  2.1464837|
|     27|   1.0|     0|      27|Salmon with Mashe...|-0.44708535|
|     12|   5.0|    28|      12|        Ceaser Salad|  0.9370873|
|     12| 